In [1]:
from __future__ import print_function
import pickle 
import argparse
import torch
import torch.nn as nn
import numpy as np
import torch.nn.functional as F
import copy
import torch.optim as optim
#from torchvision import datasets, transforms
from torch.autograd import Variable

In [2]:
trainset_labeled = pickle.load(open("train_labeled.p", "rb")) 
train_loader  = torch.utils.data.DataLoader(trainset_labeled, batch_size=64, shuffle=True, num_workers=2)

validset = pickle.load(open("validation.p", "rb"))
valid_loader = torch.utils.data.DataLoader(validset, batch_size=64, shuffle=True, num_workers=2)

Files already downloaded
3000
750


In [3]:
train_numpy = trainset_labeled.train_data.numpy()
#train_numpy = np.concatenate((train_numpy, train_numpy),0)
train_labels_numpy = trainset_labeled.train_labels.numpy()
#train_labels_numpy = np.concatenate((train_labels_numpy, train_labels_numpy),0)

In [4]:
np.zeros((3000,np.max(0,vshift),28))

NameError: name 'vshift' is not defined

In [ ]:
train_shifted = copy.copy(train_numpy)
train_labeled_shifted = copy.copy(train_labels_numpy)
train_numpy.shape
for hshift in range(-4,6,2):
    for vshift in range(-4,6,2):
        next_shift = np.concatenate((np.zeros((3000,np.max((0,vshift)),28)),
                                      train_numpy[:,np.max((0,vshift)):np.min((28,28+vshift)),:],
                                      np.zeros((3000,np.max((0,-1*vshift)),28))),1)
        
        next_shift = np.concatenate((np.zeros((3000,28,np.max((0,hshift)))),
                                      next_shift[:,:,np.max((0,hshift)):np.min((28,28+hshift))],
                                      np.zeros((3000,28,np.max((0,-1*hshift))))),2)
        
        train_shifted = np.concatenate((train_shifted, next_shift))

In [ ]:
train_shifted.shape

In [ ]:
a = np.ones(3)
a[-1:1]

In [ ]:
train_labels_numpy.shape

In [ ]:
trainset_labeled.train_data = torch.ByteTensor(train_numpy)
trainset_labeled.train_labels = torch.LongTensor(train_labels_numpy)

In [14]:
torch.sigmoid

<function torch._C.sigmoid>

In [ ]:
dir(trainset_labeled)

In [ ]:
#utility functions
def rand(x, level = 1e-3):
    return x + torch.randn(x.size()) * level

def norm_weights_2d(size):
    return BatchNorm2d(nn.Linear(size))
def norm_weights_1d(size):
    return BatchNorm2d(nn.Linear(size))

def reluN(x, level = 1e-3):
    y = x + torch.randn(x.size()) * level

In [15]:
def norm(size, dim = 1):
    if dim == 1:
        return torch.nn.BatchNorm1d(size)
    else:
        return torch.nn.BatchNorm2d(size)
    

def linear(indim, outdim):
    return nn.Linear(indim, outdim)
def var(tens):
    # The whole torch.nn module uses Parameter objects, which are essentially wrappers
    # of autograd Variables.
    return torch.nn.Parameter(tens)
def randn(size):
    return torch.randn(size)
def matmul(w1, w2):
    return w1.mm(w2)

def broad(tens, bs):
    size = list(tens.size())
    return tens.unsqueeze(0).expand(*([bs] + l))

class Net(nn.Module):
    def u(self, v, z):
        
        return self.normalize_vector(v(z))
    
    def normalize_vector(self, vect):
        vectmean = vect.mean()
        vectstd = vect.std()
        vznorm = (vect - vectmean) / vectstd
        return vznorm
    
    def normalize_vector_noise(self, vect, noise):
        vectmean = vect.mean()
        vectstd = vect.std()
        vznorm = (vect - vectmean) / vectstd
        return vznorm + noise
    
    
        
    def mu_v(u, a1, a2, a3, a4, a5):
        bs = u.size()[0]
        a_size = a1.size()[0]
        return (a1.unsqueeze(0).expand(bs,a_size) * 
            torch.sigmoid(a2.unsqueeze(0).expand(bs,a_size) * u + a3.unsqueeze(0).expand(bs,a_size)) + 
            a4.unsqueeze(0).expand(bs,a_size) * u + a5.unsqueeze(0).expand(bs,a_size))
        
    #z is lateral, u is vetrtical from above
    def g(self, z, u, a1, a2, a3, a4, a5, a6, a7, a8, a9, a10):
        return (z - mu_v(u, a1, a2, a3, a4, a5)) * mu_v(u, a6, a7, a8, a9, a10) + mu_v(u, a1, a2, a3, a4, a5)
        
        
    def __init__(self, layers = [100, 100, 100]):
        super(Net, self).__init__()
        self.layers = layers
        layers = self.layers
        self.encs = [linear(28*28, layers[0])]
        self.u = 
        self.lats = []
        self.encnorms = []
        self.purenorms = []
        self.decs = []
        self.decnorms = []
        self.a1s = []
        self.a2s = []
        self.a3s = []
        self.a4s = []
        self.a5s = []
        self.a6s = []
        self.a7s = []
        self.a8s = []
        self.a9s = []
        self.a10s = []
        
        def _add_module_to_list(name, module, module_list):
            self.__setattr__(name, module)
            module_list.append(module)

        for idx, l in enumerate(layers):
            # When setting any attribute in a Module, PyTorch will try to figure out
            # what kind of value it is: if it is a Parameter or a Module, PyTorch
            # will automatically add the Parameter or Module into the parameter/module
            # list.  I'm not sure if this is a good pattern in Python because adding
            # modules invisibly like this would cause problems if one wants to maintain
            # modules in list attributes for dynamicity (like here).  I would rather
            # require the developers to always explicitly add modules with "add_module()"
            # (I'm more convinced to avoid deriving from Module in general).
            _add_module_to_list('_encnorm%d' % idx, norm(l), self.encnorms)
            _add_module_to_list('_purenorm%d' % idx, norm(l), self.purenorms)
            if idx < len(layers) - 1:
                _add_module_to_list('_lat%d' % idx, linear(l, l), self.lats)
                _add_module_to_list('_enc%d' % idx, linear(l, layers[idx+1]), self.encs)
                _add_module_to_list('_dec%d' % idx, linear(layers[idx+1], l), self.decs)
                _add_module_to_list('_decnorm%d' % idx, norm(l), self.decnorms)
                
                _add_module_to_list('_a1s%d' % idx, nn.parameter(torch.randn(l)*1e-3), self.a1s)
                _add_module_to_list('_a2s%d' % idx, nn.parameter(torch.randn(l)*1e-3), self.a2s)
                _add_module_to_list('_a3s%d' % idx, nn.parameter(torch.randn(l)*1e-3), self.a3s)
                _add_module_to_list('_a4s%d' % idx, nn.parameter(torch.randn(l)*1e-3), self.a4s)
                _add_module_to_list('_a5s%d' % idx, nn.parameter(torch.randn(l)*1e-3), self.a5s)
                _add_module_to_list('_a6s%d' % idx, nn.parameter(torch.randn(l)*1e-3), self.a6s)
                _add_module_to_list('_a7s%d' % idx, nn.parameter(torch.randn(l)*1e-3), self.a7s)
                _add_module_to_list('_a8s%d' % idx, nn.parameter(torch.randn(l)*1e-3), self.a8s)
                _add_module_to_list('_a9s%d' % idx, nn.parameter(torch.randn(l)*1e-3), self.a9s)
                _add_module_to_list('_a10s%d' % idx, nn.parameter(torch.randn(l)*1e-3), self.a10s)
        self.batch_size = 64
        self.weights = [self.encs, self.lats, self.decs]
        

    def forward(self, x, v = 0, noise=1e-3):
        self.eps = noise
        self.batch_size = x.size()[0]
        bs = self.batch_size
        #enc= F.relu(self.enc4(F.relu(self.enc3(F.relu(self.enc2(F.relu(self.enc1(x))))))))
        corrupted = []
        corruptedout = []
        corrupted_pre_scaled = []
        for idx, l in enumerate(self.layers):
            if idx == 0:
                corrupted.append(self.encnorms[idx](self.encs[idx](x)))
                cur_noise = var(randn(corrupted[-1].size()) * self.eps)
                corrupted_pre_scaled.append(self.normalize_vector_noise(self.encs[idx](x), cur_noise))
            else:
                corrupted.append(self.encnorms[idx](self.encs[idx](corruptedout[-1])))
                cur_noise = var(randn(corrupted[-1].size()) * self.eps)
                corrupted_pre_scaled.append(self.normalize_vector_noise(self.encs[idx](corruptedout[-1]), cur_noise))
                
            corruptedout.append(F.relu(corrupted[-1] + self.encnorms[idx].weight.unsqueeze(0).expand(
                    bs, l) * cur_noise +
                    self.encnorms[idx].bias.unsqueeze(0).expand(bs, l)))
            
        encout = F.softmax(corruptedout[-1])
        
        clean = [x]
        clean_pre_scaled = []
        mu_l = []
        std_l = []
        for norm, enc in zip(self.encnorms, self.encs):
            clean_pre_scaled.append(self.normalize_vector(clean[-1]))
            mu_l.append(clean_pre_scaled[-1].mean())
            std_l.append(clean_pre_scaled[-1].std())
            clean.append(F.relu(norm(enc(clean[-1]))))
        
        #Decout starts as the output. For future iterations down decoding path,
        #Decout[-1] will contain V*Z of above decoder
        decout = [encout]
        z_hats = []
        for idx in range(len(self.layers) - 2, -1, -1):
            a1, a2, a3, a4, a5 a6, a7, a8, a9, a10 = a1s[idx], a1s[idx], a2s[idx], a3s[idx], \
                a4s[idx], a5s[idx], a6s[idx], a7s[idx], a8s[idx], a9s[idx], a10s[idx]
            
            decin = decout[-1]
            U_l = self.normalize_vector(decin)
            z_tild = corrupted_pre_scaled[idx]
            z_hat = self.g(z_tild, U_l)
            z_hats.append(z_hat - mu[idx])
            
            
            
            v = decs[idx]
                
            dec = self.decs[idx]
            lat = self.lats[idx]
            decnorm = self.decnorms[idx]
            decin.append(dec(decout[-1]) + lat(corruptedout[idx]))
            decout.append(F.relu(decin[-1] + decnorm.weight.unsqueeze(0).expand(bs, self.layers[idx]) *
                    var(randn(decin[-1].size()) * self.eps) + \
                    decnorm.bias.unsqueeze(0).expand(bs, self.layers[idx])))
        weight_reg = 0
        for w_list in self.weights:
            for w in w_list:
                weight_reg += (w.weight**2).mean()/100
        
        yhat = F.log_softmax(corruptedout[-1])
        
        encode_err = 0
        enc_weight = 1
        enc_decay = .5
        for c, d in zip(clean[-2::-1], decin):
            encode_err += enc_weight * ((c - d)**2).mean()
            enc_weight *= enc_decay
        return yhat, weight_reg + encode_err
        

model = Net([600, 300, 200, 100])
params = list(model.parameters())

if 0:
    print(model)
    print('Models has {} learnable paramater:'.format(len(params)))
    [print('parameter {} has a size of {}'.format(i+1, params[i].size())) for i in range(len(params))]

SyntaxError: invalid syntax (<ipython-input-15-1564595a6124>, line 42)

In [6]:
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [7]:
def train(epoch):
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data), Variable(target) # Wrap them in Variable 
        optimizer.zero_grad() # Zero the parameter gradients
        data = data.view(data.size()[0], 28*28)
        outputs = model(data,noise=1e-2) # Forward 
        output = outputs[0]
        US = outputs[1]
        loss = F.nll_loss(output, target) + US*.1
        loss.backward() 
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data[0]))

In [8]:
np.array(trainset_labeled)

RuntimeError: bool value of non-empty torch.ByteTensor objects is ambiguous

In [10]:
def train(epoch):
    model.train()
    
    
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data), Variable(target) # Wrap them in Variable
        optimizer.zero_grad() # Zero the parameter gradients
        data = data.view(data.size()[0], 28*28)
        data2 = data.clone()
        '''
        for flips in range(3):
            if flips == 0:
                hshift = np.random.randint(10)
                vshift = np.random.randint(10)
                data2 = copy.deepcopy(data.resize_as(torch.FloatTensor(data.size()[0], 28, 28)))
                #data2 = data.resize_as(torch.FloatTensor(data.size()[0], 28, 28))
                if 1:
                    data2[:,:,hshift:] = data2[:,:,:28-hshift]
                    data2[:,:,:hshift] = 0
                    #data2 = torch.cat((torch.FloatTensor(data.size()[0], 28,hshift), data2[:,:,hshift:]))
            if flips == 1:
                data2 = data.resize_as(torch.FloatTensor(data.size()[0], 28, 28))
                data2 = data2.transpose(1,2)
                data2 = data2.resize_as(torch.FloatTensor(data.size()[0], 784))
            if flips == 2:
                data2 = data
        '''
        outputs = model(data,noise=1e-2) # Forward 
        output = outputs[0]
        US = outputs[1]
        loss = F.nll_loss(output, target) + US*.1
        loss.backward() 
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data[0]))

In [11]:
lshift = np.random.random()
print(lshift)

0.5188185825348912


In [12]:
def test(epoch, valid_loader):
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in valid_loader:
            data, target = Variable(data, volatile=True), Variable(target)
            data = data.view(data.size()[0], 28*28)
            outputs = model(data, noise = 0)
            output = outputs[0]
            test_loss += F.nll_loss(output, target).data[0]
            pred = output.data.max(1)[1] # get the index of the max log-probability
            correct += pred.eq(target.data).cpu().sum()

    test_loss /= len(valid_loader) # loss function already averages over batch size
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(valid_loader.dataset),
        100. * correct / len(valid_loader.dataset)))

In [13]:
for epoch in range(1, 30):
    train(epoch)
    test(epoch, valid_loader)

Train Epoch: 1 [0/3000 (0%)]	Loss: 4.682650
Train Epoch: 1 [640/3000 (21%)]	Loss: 3.249107
Train Epoch: 1 [1280/3000 (43%)]	Loss: 2.794274
Train Epoch: 1 [1920/3000 (64%)]	Loss: 2.401498
Train Epoch: 1 [2560/3000 (85%)]	Loss: 2.018213

Test set: Average loss: 1.7966, Accuracy: 9049/10000 (90%)

Train Epoch: 2 [0/3000 (0%)]	Loss: 1.722873


Process Process-5:


KeyboardInterrupt: 

Traceback (most recent call last):
  File "/home/lee/anaconda3/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
Process Process-6:
  File "/home/lee/anaconda3/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
Traceback (most recent call last):
  File "/home/lee/anaconda3/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()


In [80]:
a.RandomHorizontalFlip

NameError: name 'a' is not defined

In [105]:
import torchvision

In [110]:
torch.from_numpy(arr)


 1
 1
 1
[torch.DoubleTensor of size 3]

In [109]:
arr = np.ones(3)

In [115]:
a.asNumpyarray()

AttributeError: asNumpyarray

In [114]:
a

Variable containing:
-0.4242 -0.4242 -0.4242  ...  -0.4242 -0.4242 -0.4242
-0.4242 -0.4242 -0.4242  ...  -0.4242 -0.4242 -0.4242
-0.4242 -0.4242 -0.4242  ...  -0.4242 -0.4242 -0.4242
          ...             ⋱             ...          
-0.4242 -0.4242 -0.4242  ...  -0.4242 -0.4242 -0.4242
-0.4242 -0.4242 -0.4242  ...  -0.4242 -0.4242 -0.4242
-0.4242 -0.4242 -0.4242  ...  -0.4242 -0.4242 -0.4242
[torch.FloatTensor of size 64x784]

In [116]:
a = list(range(784,-1,-1))

In [129]:
dataarr[:]

Variable containing:
-0.4242
-0.4242
-0.4242
-0.4242
-0.4242
-0.4242
-0.4242
-0.4242
-0.4242
-0.4242
-0.4242
-0.4242
-0.4242
-0.4242
-0.4242
-0.4242
-0.4242
-0.4242
-0.4242
-0.4242
-0.4242
-0.4242
-0.4242
-0.4242
-0.4242
-0.4242
-0.4242
-0.4242
-0.4242
-0.4242
-0.4242
-0.4242
-0.4242
-0.4242
-0.4242
-0.4242
-0.4242
-0.4242
-0.4242
-0.4242
-0.4242
-0.4242
-0.4242
-0.4242
-0.4242
-0.4242
-0.4242
-0.4242
-0.4242
-0.4242
-0.4242
-0.4242
-0.4242
-0.4242
-0.4242
-0.4242
-0.4242
-0.4242
-0.4242
-0.4242
-0.4242
-0.4242
-0.4242
-0.4242
[torch.FloatTensor of size 64]

In [130]:
torch.image

AttributeError: module 'torch' has no attribute 'image'

In [138]:
dataarr.scale

AttributeError: scale

In [153]:
torchvision.transforms

<module 'torchvision.transforms' from '/home/lee/anaconda3/lib/python3.5/site-packages/torchvision-0.1.6-py3.5.egg/torchvision/transforms.py'>